## Example From Hands On ML Book

### Look At The Big Picture

It helps to have a checklist of what you will go through and do. Example: 
* frame problem and look at big picture
* get data
* explore the data to get insights
* prepare the data to better expose the underlying patterns to ML algorithms
* explore many different models and shortlist the best ones
* fine tune your models and combine them into a great solution
* present your solution
* launch, monitor, and maintain your system


Big picture in this case is what is the objective? even though they might say they want a model, what they use it for is important and will have impact on how the problem is framed, what model is selected, performance metrics used and how much effort is spent tweaking the model.

For this dataset and model we happen to know that the prediction of housing prices will be fed to another ML system with other signals in order to determine whether the area is worth looking into and the model accuracy will have a direct impact on company revenue.

continue from here on next session: https://learning.oreilly.com/library/view/hands-on-machine-learning/9781098125967/ch02.html#:-:text=On%20the%20other,system%E2%80%99s%20performance%20drops.

### Get The Data

### Explore and visualize the data to gain insights

### Prepare the data for machine learning algorithms

### Select Model and train it

### Fine tune your model

### Present Solution

### Launch, Monitor, Maintain Your System